In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io

from dosipy.utils.viz import fig_config, save_fig, set_colorblind

# head

In [ ]:
frequencies = {0: 2.4, 3: 5, 5: 10}
distances = [2.5, 5, 10, 50, 150]
cp = 7
# simulation
adult = pd.read_csv(os.path.join('data', 'adult_head.csv'))
adult = adult[adult['col_points'] == cp]
child = pd.read_csv(os.path.join('data', 'child_head.csv'))
child = child[child['col_points'] == cp]
# stochastic analysis
mean_adult_mat = scipy.io.loadmat(os.path.join('data', f'adult_M_SC{cp}'))
var_adult_mat = scipy.io.loadmat(os.path.join('data', f'adult_V_SC{cp}'))
mean_child_mat = scipy.io.loadmat(os.path.join('data', f'child_M_SC{cp}'))
var_child_mat = scipy.io.loadmat(os.path.join('data', f'child_V_SC{cp}'))

In [ ]:
nrows = 2
ncols = 3
fig_config(latex=True, line_width=2.5, marker_size=7, nrows=nrows, ncols=ncols)
fig = plt.figure()
axs = fig.subplots(nrows, ncols, sharex=True)
for icol, (f_key, f) in enumerate(frequencies.items()):
    mu_adult = mean_adult_mat[f'M_SC{cp}'][f_key]
    mu_child = mean_child_mat[f'M_SC{cp}'][f_key]
    var_adult = var_adult_mat[f'V_SC{cp}'][f_key]
    var_child = var_child_mat[f'V_SC{cp}'][f_key]
    f_cond_adult = adult['frequency [GHz]'] == f
    y_ref_adult = adult[f_cond_adult]['sPDn_pln [W/m2]'].unique()
    f_cond_child = child['frequency [GHz]'] == f
    y_ref_child = child[f_cond_child]['sPDn_pln [W/m2]'].unique()
    assert (y_ref_adult == y_ref_child).all()
    y_ref = y_ref_child.copy()
    child_adult_perc_diff = (mu_child - mu_adult) / y_ref * 100
    adult_planar_perc_diff = (mu_adult - y_ref) / y_ref * 100
    child_planar_perc_diff = (mu_child - y_ref) / y_ref * 100
    
    axs[0, icol].plot(distances, y_ref, 'k-',
                      label='planar',
                      marker='o',
                      markeredgecolor='k',
                      markerfacecolor='k',
                      markeredgewidth=1.5)
    axs[0, icol].errorbar(distances, mu_adult, yerr=3*np.sqrt(var_adult),
                          c='r', ls='--',
                          label=r'$\langle \text{adult} \rangle$, 99\% CI')
    axs[0, icol].errorbar(distances, mu_child, yerr=3*np.sqrt(var_child),
                          c='b', ls=':',
                          label=r'$\langle \text{child} \rangle$, 99\% CI')
    axs[0, icol].set_xscale('log')
    axs[0, icol].set_xticks(ticks=distances, labels=distances)
    axs[0, icol].set_yticks(ticks=(0, 3, 6, 9), labels=(0, 3, 6, 9))
    axs[0, icol].set_ylim([-0.9, 9.9])
    axs[0, icol].set_title(f'$f = {f}$ GHz')
    if icol==0:
        axs[0, icol].set_ylabel('$sPDn$ [W/m$^2$]')
    axs[0, icol].legend()
    axs[0, icol].grid()
                     
    axs[1, icol].plot(distances, child_adult_perc_diff, 'k-',
                      label=r'$\langle$child$\rangle$-$\langle$adult$\rangle$',
                      marker='o',
                      markeredgecolor='k',
                      markerfacecolor='k',
                      markeredgewidth=1.5)
    axs[1, icol].plot(distances, adult_planar_perc_diff, 'r--',
                      label=r'$\langle$adult$\rangle$-planar',
                      marker='x',
                      markeredgecolor='r',
                      markerfacecolor='r',
                      markeredgewidth=1.5)
    axs[1, icol].plot(distances, child_planar_perc_diff, 'b:',
                      label=r'$\langle$child$\rangle$-planar',
                      marker='d',
                      markeredgecolor='b',
                      markeredgewidth=1.5)
    axs[1, icol].set_xscale('log')
    axs[1, icol].set_xticks(ticks=distances, labels=distances)
    axs[1, icol].set_yticks(ticks=(0, 4, 8, 12), labels=(0, 4, 8, 12))
    axs[1, icol].set_ylim([-2.2, 13.2])
    axs[1, icol].set_xlabel('$d$')
    if icol==0:
        axs[1, icol].set_ylabel('\% difference')
    axs[1, icol].legend()
    axs[1, icol].grid()

fig.tight_layout();

fname = 'head_stochastic_analysis'
save_fig(fig, os.path.join('figures', fname))

# curvature effect

In [ ]:
frequencies = [2.4, 5, 10]
distances = [2.5, 5, 10, 50, 150]
radii = [1.8, 18]
curvature = pd.read_csv(os.path.join('data', 'curvature.csv'))

In [ ]:
nrows = 2
ncols = 3
fig_config(latex=True, line_width=2.5, marker_size=7, nrows=nrows, ncols=ncols)
fig = plt.figure()
axs = fig.subplots(nrows, ncols, sharex=True)
for icol, f in enumerate(frequencies):
    f_cond = curvature['frequency [GHz]'] == f
    extracted_curvature = curvature[f_cond]
    r_cond_small = extracted_curvature['radius [cm]'] == min(radii)
    r_cond_big = extracted_curvature['radius [cm]'] == max(radii)
    y_small = extracted_curvature[r_cond_small]['sPDn_sph [W/m2]'].to_numpy()
    y_big = extracted_curvature[r_cond_big]['sPDn_sph [W/m2]'].to_numpy()
    y_ref_1 = extracted_curvature[r_cond_small]['sPDn_pln [W/m2]'].to_numpy()
    y_ref_2 = extracted_curvature[r_cond_big]['sPDn_pln [W/m2]'].to_numpy()
    assert (y_ref_1 == y_ref_2).all()
    y_ref = y_ref_1.copy()
    small_big_perc_diff = (y_small - y_big) / y_ref * 100
    big_planar_perc_diff = (y_big - y_ref) / y_ref * 100
    small_planar_perc_diff = (y_small - y_ref) / y_ref * 100
    
    axs[0, icol].plot(distances, y_ref, 'k-',
                      label='planar',
                      marker='o',
                      markeredgecolor='k',
                      markerfacecolor='k',
                      markeredgewidth=1.5)
    axs[0, icol].plot(distances, y_big, 'r--',
                      label=r'$r_\text{max}$',
                      marker='x',
                      markeredgecolor='r',
                      markeredgewidth=1.5)
    axs[0, icol].plot(distances, y_small, 'b:',
                      label=r'$r_\text{min}$',
                      marker='d',
                      markeredgecolor='b',
                      markerfacecolor='b',
                      markeredgewidth=1.5)
    axs[0, icol].set_xscale('log')
    axs[0, icol].set_xticks(ticks=distances, labels=distances)
    axs[0, icol].set_yticks(ticks=(0, 3, 6, 9), labels=(0, 3, 6, 9))
    axs[0, icol].set_ylim([-0.9, 9.9])
    axs[0, icol].set_title(f'$f = {f}$ GHz')
    if icol==0:
        axs[0, icol].set_ylabel('$sPDn$ [W/m$^2$]')
    axs[0, icol].legend()
    axs[0, icol].grid()
                     
    axs[1, icol].plot(distances, small_big_perc_diff, 'k-',
                      label=r'$r_\text{min}$-$r_\text{max}$',
                      marker='o',
                      markeredgecolor='k',
                      markerfacecolor='k',
                      markeredgewidth=1.5)
    axs[1, icol].plot(distances, big_planar_perc_diff, 'r--',
                      label=r'$r_\text{max}$-planar',
                      marker='x',
                      markeredgecolor='r',
                      markerfacecolor='r',
                      markeredgewidth=1.5)
    axs[1, icol].plot(distances, small_planar_perc_diff, 'b:',
                      label=r'$r_\text{min}$-planar',
                      marker='d',
                      markeredgecolor='b',
                      markeredgewidth=1.5)
    axs[1, icol].set_xscale('log')
    axs[1, icol].set_xticks(ticks=distances, labels=distances)
    axs[1, icol].set_yticks(ticks=(-12, 0, 12, 24), labels=(-12, 0, 12, 24))
    axs[1, icol].set_ylim([-14.4, 26.4])
    axs[1, icol].set_xlabel('$d$')
    if icol==0:
        axs[1, icol].set_ylabel('\% difference')
    axs[1, icol].legend()
    axs[1, icol].grid()

fig.tight_layout();

fname = 'min_max_curvature_analysis'
save_fig(fig, os.path.join('figures', fname))

# build figure

In [ ]:
# tba